# Telemetry-to-Insight: Full Pipeline Demo

End-to-end: load telemetry with cuDF, retrieve filtered data, send to NIM for natural-language answers.

**Colab:** You need NIM running on GKE with a public URL. Paste your NIM URL in the Setup cell below. See "Deploy NIM on GKE" at the end of this notebook.

<a href="https://colab.research.google.com/github/KarthikSriramGit/Project-Insight/blob/main/notebooks/03_query_telemetry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Configure runtime first:** Runtime > Change runtime type > Hardware accelerator: **GPU** or **None** > Save.

In [2]:
# Colab setup: clone repo and install dependencies (run this cell first)
try:
    import google.colab
    get_ipython().system("git clone -q https://github.com/KarthikSriramGit/Project-Insight.git")
    get_ipython().run_line_magic("cd", "Project-Insight")
    get_ipython().system("pip install -q -r requirements.txt")
except Exception:
    pass

/content/Project-Insight


## Setup

**Edit the cell below:** Replace `YOUR_EXTERNAL_IP` with your GKE NIM LoadBalancer IP (from `kubectl get svc -n nim`).

In [3]:
import os
import sys
import subprocess
from pathlib import Path

ROOT = Path(".").resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# Paste your GKE NIM URL here (e.g. http://34.123.45.67:8000)
NIM_BASE_URL = os.environ.get("NIM_BASE_URL", "http://136.119.171.193:8000")

data_path = ROOT / "data" / "synthetic" / "fleet_telemetry.parquet"
if not data_path.exists():
    subprocess.run([
        "python", "data/synthetic/generate_telemetry.py",
        "--rows", "100000", "--output-dir", "data/synthetic", "--format", "parquet",
    ], check=True, cwd=str(ROOT))

from src.query.engine import TelemetryQueryEngine

engine = TelemetryQueryEngine(
    data_path=str(data_path),
    nim_base_url=NIM_BASE_URL,
    max_context_rows=500,
)
print(f"Engine ready. NIM URL: {NIM_BASE_URL}")

Engine ready. NIM URL: http://136.119.171.193:8000


In [4]:
answer = engine.query("What is the max brake pressure across all vehicles?", sensor_type="can")
print(answer)

Unfortunately, the provided telemetry data does not contain enough information to determine the maximum brake pressure across all vehicles. The "brake_pressure_pct" column only contains a percentage value and not an actual pressure value. Therefore, it is not possible to determine the maximum brake pressure.
